<a href="https://colab.research.google.com/github/lilphonsdale/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'. To attempt to optimize the model, let's also drop the 'Status' column.
application_df = application_df.drop(['EIN','NAME','STATUS'],axis=1)

In [6]:
# Choose a cutoff value and create a list of application types to be replaced. To optimize the model, let's try having fewer bins.
# use the variable name `application_types_to_replace`

other_applications = application_df["APPLICATION_TYPE"].value_counts().loc[lambda x : x<1000]

application_types_to_replace = [other_applications.index]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Choose a cutoff value and create a list of classifications to be replaced. Now we will have 6 bins in both Application Type and Classification. 
# use the variable name `classifications_to_replace`
other_classifications = application_df["CLASSIFICATION"].value_counts().loc[lambda x : x<1000]
classifications_to_replace = [other_classifications.index]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_df = pd.get_dummies(data=application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])

In [9]:
# Split our preprocessed data into our features and target arrays
X = numeric_df.drop(columns="IS_SUCCESSFUL").values
y = numeric_df.IS_SUCCESSFUL.values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer. In order to optimize the model, let's try adding another hidden layer, and changing the activation function to Tanh, which should be good for a classification.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3200      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 10)                310       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5,951
Trainable params: 5,951
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
# Train the model - we'll use slightly fewer epochs this time. 
fit_model = nn.fit(X_train_scaled,y_train,epochs=80)

Epoch 1/80
804/804 [==============================] - 3s 2ms/step - loss: 0.5678 - accuracy: 0.7238
Epoch 2/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5563 - accuracy: 0.7306
Epoch 3/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5536 - accuracy: 0.7310
Epoch 4/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5520 - accuracy: 0.7332
Epoch 5/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5502 - accuracy: 0.7339
Epoch 6/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7332
Epoch 7/80
804/804 [==============================] - 2s 3ms/step - loss: 0.5485 - accuracy: 0.7342
Epoch 8/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5478 - accuracy: 0.7348
Epoch 9/80
804/804 [==============================] - 1s 2ms/step - loss: 0.5472 - accuracy: 0.7351
Epoch 10/80
804/804 [==============================] - 2s 2ms/step - loss: 0.5470 - accuracy: 0.7350

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5619 - accuracy: 0.7237 - 1s/epoch - 5ms/step
Loss: 0.5618644952774048, Accuracy: 0.7237317562103271


In [15]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")